# Visualizations

In [160]:
import altair as alt
import pandas as pd

In [161]:
# open complete csv
complete = pd.read_csv('../StarTrekNextGenScriptData/complete_data.csv')
complete.columns = ['index', 'character', 'quote', 'scene', 'location', 'view',
       'episode', 'date', 'series', 'file']

complete['character'] = complete['character'].apply(lambda text: " ".join(str(text).split()))
complete['quote']=complete['quote'].apply(lambda text: " ".join(text.split()))

In [162]:
# split the data
ds9 = complete[complete.series == 'Deep Space Nine']
tng= complete[complete.series == 'The Next Generation']

In [163]:
tng_counts = tng.groupby(['file','episode','character']).count()['quote'].reset_index()
tng_counts['pct_q_ep']=tng_counts['quote']/tng_counts.groupby('episode')['quote'].transform('sum')
tng_counts['character'] = tng_counts['character'].apply(lambda x: x[0]+x[1:].lower())

top_10 = list(tng_counts.groupby('character').sum()['quote'].reset_index().sort_values('quote',ascending=False)[:10]['character'])


tng_filtered = tng_counts[tng_counts.character.isin(top_10)]
tng_filtered

,file,episode,character,quote,pct_q_ep
3,102.txt,Encounter at Farpoint,Beverly,28,0.038674
6,102.txt,Encounter at Farpoint,Data,60,0.082873
8,102.txt,Encounter at Farpoint,Geordi,6,0.008287
16,102.txt,Encounter at Farpoint,Picard,208,0.287293
17,102.txt,Encounter at Farpoint,Riker,135,0.186464
...,...,...,...,...,...
2671,277.txt,All Good Things...,Picard,328,0.379191
2673,277.txt,All Good Things...,Riker,58,0.067052
2674,277.txt,All Good Things...,Tasha,32,0.036994
2676,277.txt,All Good Things...,Troi,43,0.049711


In [164]:
alt.Chart(tng_filtered).mark_bar().encode(
    #x =  alt.X('character:N'),
    y= alt.X('pct_q_ep'),
    x=alt.Y('character:N'),
    color=alt.Color('character'),
    size=alt.Size('pct_q_ep:Q'),
    order=alt.Order('ep_id')
)

ValueError: Unable to determine data type for the field "ep_id"; verify that the field name is not misspelled. If you are referencing a field from a transform, also confirm that the data type is specified correctly.

alt.Chart(...)

In [165]:
unique_values = tng_filtered['episode'].unique()
mapping = {val: i for i, val in enumerate(unique_values)}
tng_filtered['ep_id'] = tng_filtered['episode'].map(mapping)

max=tng_filtered.pct_q_ep.max()

/var/folders/r0/2fgtj8y934zddv46dgfbq1xw0000gn/T/ipykernel_95890/1649851097.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tng_filtered['ep_id'] = tng_filtered['episode'].map(mapping)


In [166]:
tng_filtered.columns=['file', 'Episode', 'character', 'Number of Quotes', 'pct_q_ep', 'ep_id']

In [167]:
top_10

['Picard',
 'Riker',
 'Data',
 'Geordi',
 'Worf',
 'Beverly',
 'Troi',
 'Wesley',
 'Pulaski',
 'Tasha']

In [173]:
alt.Chart(tng_filtered).transform_calculate(
    x2 = 'datum.pct_q_ep * -1'
).mark_rect(
).encode(
    #x =  alt.X('character:N'),
    y= alt.Y('ep_id:N',axis=None),
    x=alt.X('pct_q_ep', type="quantitative",axis=None,scale=alt.Scale(domain=[-max,max])),
    x2=alt.X2('x2'),
    order=alt.Order('ep_id',sort='descending'),
    color = alt.Color('character:N',legend=None),
    #tooltip=['Episode:N','Number of Quotes']
).properties(width=30,height=600).facet(
    column=alt.Column('character:N',sort=top_10,title=None),
    padding=0,
    spacing = 0,
).configure_view(stroke=None).resolve_axis(y='shared').properties(title='Star Trek: The Next Generation')

alt.FacetChart(...)